In [9]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 111.8 MB/s eta 0:00:00


In [1]:
from sklearn.datasets import load_wine
x, y = load_wine(return_X_y = True)

In [2]:
x = x[:59+71]
y = y[:59+71]

In [3]:
from sklearn.model_selection import train_test_split
x_tr, x_test, y_tr, y_test = train_test_split(x, y, train_size = 0.9, shuffle = True, stratify = y)

In [4]:
from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()
x_tr = scaler.fit_transform(x_tr)

In [5]:
import numpy as np
x_test = scaler.transform(x_test)
x_test = np.clip(x_test, 0, 1)

In [10]:
import pennylane as qml
ncubits = 4
dev = qml.device("lightning.qubit", wires = ncubits)
@qml.qnode(dev)
def circuito_kernel(a, b):
    qml.AmplitudeEmbedding(
        b, wires = range(ncubits), pad_with = 0, normalize = True
    )
    qml.adjoint(qml.AmplitudeEmbedding(
        a, wires = range(ncubits), pad_with = 0, normalize = True
    ))
    return qml.probs(wires = range(ncubits))


In [7]:
def qkernel(A, B): # A, B son arrays de vectores
    return np.array([[circuito_kernel(a, b)[0] for b in B] for a in A])
    # Devuelve una matriz X donde X[i,j] = A[i] * B[j]

In [11]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [12]:
svm = SVC(kernel = qkernel).fit(x_tr, y_tr)
print(accuracy_score(svm.predict(x_test), y_test))

0.9230769230769231


In [13]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 8)
xs_tr = pca.fit_transform(x_tr)
xs_test = pca.transform(x_test)

In [14]:
ncubits = 8
dev = qml.device("lightning.qubit", wires = ncubits)

@qml.qnode(dev)
def circuito_kernel(a, b):
    qml.AngleEmbedding(b, wires = range(ncubits))
    qml.adjoint(qml.AngleEmbedding(a, wires = range(ncubits)))
    return qml.probs(wires = range(ncubits))


In [15]:
svm = SVC(kernel = qkernel).fit(xs_tr, y_tr)
print(accuracy_score(svm.predict(xs_test), y_test))
print(accuracy_score(svm.predict(xs_tr), y_tr))

1.0
0.9829059829059829


In [16]:
from itertools import combinations
def CodificadorZZ(ncubits, datos):
    ncarga = min(len(datos), ncubits)
    for i in range(ncarga):
        qml.Hadamard(wires = i)
        qml.RZ(2.0 * datos[i], wires = i)
    for par in list(combinations(range(ncubits), 2)):
        q0 = par[0]
        q1 = par[1]
        qml.CNOT(wires = [q0, q1])
        qml.RZ(2.0 * (np.pi - datos[q0]) * (np.pi - datos[q1]), wires = q1)
        qml.CNOT(wires = [q0, q1])

In [17]:
ncubits = 8
dev = qml.device("lightning.qubit", wires = ncubits)

@qml.qnode(dev)
def circuito_kernel(a, b):
    CodificadorZZ(ncubits, b)
    qml.adjoint(CodificadorZZ)(ncubits, a)
    return qml.probs(wires = range(ncubits))

In [18]:
svm = SVC(kernel = qkernel).fit(xs_tr, y_tr)
print(accuracy_score(svm.predict(xs_test), y_test))
print(accuracy_score(svm.predict(xs_tr), y_tr))

0.9230769230769231
1.0


In [1]:
!pip install git+https://github.com/pycaret/pycaret.git@master

  Cloning https://github.com/pycaret/pycaret.git (to revision master) to /tmp/pip-req-build-wm7impic
  Running command git clone --filter=blob:none --quiet https://github.com/pycaret/pycaret.git /tmp/pip-req-build-wm7impic
  Resolved https://github.com/pycaret/pycaret.git to commit 58ec3c282d58e94727f9d5b77b49f241e9103ab3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!pip install cirq

  Using cached matplotlib-3.10.7-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached scipy-1.16.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (62 kB)
Using cached matplotlib-3.10.7-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (8.7 MB)
Using cached scipy-1.16.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (35.7 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.5
    Uninstalling matplotlib-3.7.5:
      Successfully uninstalled matplotlib-3.7.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pycaret 3.4.0 requires matplotlib<3.8.0, but you have matplotlib 3.10.7 which is incompatible.
pyca

In [4]:
# -----------------------------------------------------------
# PASO 1: INSTALACIÓN DE LIBRERÍAS (Asegúrate de que PyCaret esté instalado)
# pip install pycaret
# -----------------------------------------------------------

# -----------------------------------------------------------
# PASO 2: IMPORTACIÓN DE LIBRERÍAS CLÁSICAS Y PYCARET
# -----------------------------------------------------------
from pycaret.classification import setup, get_config
from sklearn.datasets import load_wine
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import cirq # No usado en qsvm, pero mantenido por contexto
import pennylane as qml
from itertools import combinations
import pandas as pd

# -----------------------------------------------------------
# PASO 3: CARGA DE DATOS Y PREPARACIÓN (USANDO PYCARET)
# -----------------------------------------------------------

# Cargar el dataset Wine completo (sin el recorte original)
data = load_wine(as_frame=True)
df = data.frame
# Recortar el dataset para las clases 0 y 1, como en el cuaderno original
df = df[df['target'].isin([0, 1])].reset_index(drop=True)

# Usar PyCaret para automatizar el pre-procesamiento y la división de datos
# - target='target': Especifica la columna de la clase.
# - train_size=0.9: Divide los datos 90% entrenamiento / 10% prueba.
# - normalize=True: Escala los datos (similar a MaxAbsScaler o MinMax, por defecto a Z-Score).
# - session_id: Fija la semilla para reproducibilidad.
s = setup(
    data=df,
    target='target',
    train_size=0.9,
    normalize=True,
    normalize_method='maxabs', # Especificamos MaxAbsScaler para alinearnos al cuaderno original
    session_id=123
)



,Description,Value
0,Session id,123
1,Target,target
2,Target type,Binary
3,Original data shape,"(130, 14)"
4,Transformed data shape,"(130, 14)"
5,Transformed train set shape,"(117, 14)"
6,Transformed test set shape,"(13, 14)"
7,Numeric features,13
8,Preprocess,True
9,Imputation type,simple


In [9]:
get_config()

{'USI',
 'X',
 'X_test',
 'X_test_transformed',
 'X_train',
 'X_train_transformed',
 'X_transformed',
 '_available_plots',
 '_ml_usecase',
 'data',
 'dataset',
 'dataset_transformed',
 'exp_id',
 'exp_name_log',
 'fix_imbalance',
 'fold_generator',
 'fold_groups_param',
 'fold_shuffle_param',
 'gpu_n_jobs_param',
 'gpu_param',
 'html_param',
 'idx',
 'is_multiclass',
 'log_plots_param',
 'logging_param',
 'memory',
 'n_jobs_param',
 'pipeline',
 'seed',
 'target_param',
 'test',
 'test_transformed',
 'train',
 'train_transformed',
 'variable_and_property_keys',
 'variables',
 'y',
 'y_test',
 'y_test_transformed',
 'y_train',
 'y_train_transformed',
 'y_transformed'}

In [12]:
x_tr = get_config('X_train_transformed')
y_tr = get_config('y_train')
x_test = get_config('X_test_transformed')
y_test = get_config('y_test')

In [15]:
# -----------------------------------------------------------
# PASO 4: REDUCCIÓN DE DIMENSIONALIDAD (MANUAL COMO EN EL CUADERNO ORIGINAL)
# Aunque PyCaret puede hacer PCA, aquí mantenemos la lógica manual para seguir el ejemplo original.
# -----------------------------------------------------------
from sklearn.decomposition import PCA
# Las 13 características originales son las columnas de X_train/X_test.
# Vamos a reducir a 4 para usar AmplitudeEmbedding (2^4 = 16) o a 8 (para AngleEmbedding)
ncubits = 4
n_features_orig = x_tr.shape[1]

# Usamos PCA para reducir la dimensionalidad a ncubits para el circuito
if n_features_orig > ncubits:
    pca = PCA(n_components=ncubits)
    x_tr = pca.fit_transform(x_tr)
    x_test = pca.transform(x_test)
else:
    print(f"No se requiere PCA. Usando {n_features_orig} características.")

No se requiere PCA. Usando 4 características.


In [17]:

# -----------------------------------------------------------
# PASO 5: MODELADO CUÁNTICO (QSVM) - Igual que en el cuaderno qsvm.ipynb
# -----------------------------------------------------------

# Definición del Kernel Cuántico (Amplitude Embedding - requiere normalización de vectores)
dev = qml.device("lightning.qubit", wires=ncubits)

@qml.qnode(dev)
def circuito_kernel(a, b):
    # Aseguramos la normalización requerida para AmplitudeEmbedding
    norm_b = np.linalg.norm(b)
    if norm_b > 0:
        b = b / norm_b

    norm_a = np.linalg.norm(a)
    if norm_a > 0:
        a = a / norm_a

    qml.AmplitudeEmbedding(
        b, wires=range(ncubits), pad_with=0, normalize=False
    )
    qml.adjoint(qml.AmplitudeEmbedding(
        a, wires=range(ncubits), pad_with=0, normalize=False
    ))
    return qml.probs(wires=range(ncubits))

# Función del Kernel (Producto Interno Cuadrado)
def qkernel(A, B):
    # Esta función se espera que sea lenta, por lo que es mejor usar un dataset pequeño
    return np.array([[circuito_kernel(a, b)[0] for b in B] for a in A])


In [18]:

# -----------------------------------------------------------
# PASO 6: ENTRENAMIENTO Y EVALUACIÓN
# -----------------------------------------------------------

# Entrenar el SVM Clásico con el Kernel Cuántico (QSVM)
print("\n--- Entrenando QSVM con datos pre-procesados por PyCaret ---")
svm = SVC(kernel=qkernel).fit(x_tr, y_tr.astype(int))

# Predicción y Evaluación
y_pred_test = svm.predict(x_test)
y_pred_tr = svm.predict(x_tr)

acc_test = accuracy_score(y_test.astype(int), y_pred_test)
acc_tr = accuracy_score(y_tr.astype(int), y_pred_tr)

print(f"Precisión en Test (QSVM): {acc_test:.4f}")
print(f"Precisión en Entrenamiento (QSVM): {acc_tr:.4f}")


--- Entrenando QSVM con datos pre-procesados por PyCaret ---
Precisión en Test (QSVM): 0.6154
Precisión en Entrenamiento (QSVM): 0.6496
